In [5]:
import sys

sys.path.insert(0, 'D:\\bi12year3\intern\ictlab\imgonnacrylmao')
from mod import display_ano, play_ano, process_ano, sort_ano, PATHS, CONFIG, CustomBoundingBoxAnnotator, add_check, check_life, check_dir
from obj_dect import run_main

In [40]:
ano = {2: [[301, 102, 336, 123], 174, [186, 6], [1793, 60]], 
       217: [[310, 103, 334, 120], 81, [2634, 88], [2696, 90]], 
       403: [[583, 155, 614, 180], 140, [4872, 162], [26819, 894]], 
       864: [[583, 155, 615, 180], 769, [10362, 345], [26819, 894]], 
       1148: [[583, 156, 614, 181], 77, [13416, 447], [14243, 475]], 
       1294: [[336, 55, 355, 68], 68, [14865, 496], [26819, 894]], 
       1302: [[584, 154, 613, 179], 346, [15144, 505], [26819, 894]], 
       1440: [[335, 55, 354, 68], 77, [16338, 545], [26819, 894]], 
       1477: [[336, 54, 356, 68], 96, [16884, 563], [17087, 570]], 
       1549: [[582, 154, 614, 180], 288, [17589, 586], [26819, 894]], 
       1706: [[335, 55, 355, 69], 89, [19341, 645], [19652, 655]], 
       1804: [[335, 55, 355, 69], 62, [20493, 683], [20597, 687]], 
       1845: [[336, 55, 357, 69], 68, [21249, 708], [26819, 894]]}

In [6]:
vid_no = 33
mode = "train"
fr_skp = 1
life = 60
err = 1
ano_range = 45
ano_time = 200
weights = [err, life, fr_skp, ano_range, ano_time]

In [23]:
# run_main(vid_no, mode, show= True, save= True,
#          frame_skip= fr_skp,
#          life_time= life,
#          errors= err,
#          )

In [1]:
done = sort_ano(anomalies= ano, 
                    anomaly_range= ano_range, 
                    anomaly_min_time= ano_time)
done

NameError: name 'sort_ano' is not defined

In [25]:
print("********************************")
print("THIS IS VID NO: ", vid_no)
print("###################################")

# processes = process_ano(vid_no, mode, 
#                         anomaly_range= ano_range, 
#                         anomaly_min_time= ano_time)
# play_ano(vid_no, mode, processes, anomaly_range= ano_range )
play_ano(vid_no, mode, done)

********************************
THIS IS VID NO:  33
###################################
Number of anomalies:  2
Anomaly 1
(403, [[583, 155, 614, 180], [4872, 162], [26819, 894]])
Anomaly 2
(1294, [[336, 55, 355, 68], [14865, 496], [26819, 894]])


In [13]:
from ultralytics import YOLO
import supervision as sv
import time
from collections import OrderedDict
import cv2

def deploy_model(source, weights, save= False):
    """
    weight = [error, life, fr_skp, ano_range, min_time]
    """
    # weight = (error, life, fr_skp, ano_range, min_time)
    errors, life_time, frame_skip, anomalies_range, min_time= weights
    classes = [0, 1, 2, 3, 5, 7]

    tracker_memo = CONFIG['tracker_memo']
    model = YOLO(PATHS['general'] + '\\models\\yolov8n.pt')

    # vid = cv2.VideoCapture(0) 

    video_info = sv.VideoInfo.from_video_path(video_path= source) 
    # fps = 30
    fps = video_info.fpsthickness = sv.calculate_optimal_line_thickness(resolution_wh= video_info.resolution_wh)
    thickness = sv.calculate_optimal_line_thickness(resolution_wh= video_info.resolution_wh)
    bounding_box_annotator = CustomBoundingBoxAnnotator(thickness= thickness)

    text_scale = sv.calculate_optimal_text_scale(resolution_wh= video_info.resolution_wh)

    label_annotator = sv.LabelAnnotator(text_scale= text_scale, text_thickness= thickness, text_padding= thickness)

    frame_generator = sv.get_video_frames_generator(source)

    # tracking
    byte_track = sv.ByteTrack(frame_rate= fps, lost_track_buffer= 120)

    smother = sv.DetectionsSmoother()
    
    vehicle_id = OrderedDict()

    anomalies = {}
    frame_no = 0

    start = time.time()
    
    # vid = cv2.VideoCapture(0) 
  
    # while(True): 
    #     ret, frame = vid.read() 
    for frame in frame_generator:
        timer = frame_no/fps
        timer = int(round(timer, 0))
        print(f"vid no: {vid_no}")
        print(frame_no)
        print(timer)
        if frame_no >= 0:
        # if timer > 0:
            vid_time = [frame_no, timer]
            
            result = model.predict(frame, classes= classes, conf= 0.5, device= 'cuda:0')[0]
            detections = sv.Detections.from_ultralytics(result).with_nms(threshold= CONFIG['nms'])
            detections = byte_track.update_with_detections(detections= detections)
            detections = smother.update_with_detections(detections)
            if frame_no % frame_skip == 0:#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<qqqqqqqqqqq
                vehicle_id = add_check(detections, vehicle_id, anomalies, vid_time, tracker_memo, errors)

            anomalies, vehicle_id = check_life(vehicle_id, anomalies, vid_time, life_time)
            print(anomalies)
            frame_no += 1

            annotated_frame = frame.copy()

            annotated_frame = bounding_box_annotator.annotate(
                scene= annotated_frame,
                detections= detections,
                anomalies= anomalies,
                custom_color_lookup= None
            )
            annotated_frame = cv2.putText(
                    annotated_frame, f"frame: {frame_no} \\time: {timer}", 
                    org= (20, 50), 
                    fontFace= cv2.FONT_HERSHEY_COMPLEX,
                    fontScale= 1, 
                    color= (0, 255, 0), 
                    thickness= 2)
                
            cv2.imshow(f"annotated_frame{vid_no}", annotated_frame)

            if cv2.waitKey(1) == ord("q"):
                break

            elif cv2.waitKey(1) == ord('m'):
                output = sort_ano(anomalies, weights[3], weights[4])
                ano_file_path = PATHS['general'] + "output\\output.txt"
                text_file = open(ano_file_path, 'w')
                for i in output.items():
                    text_file.write(f"{str(i)}\n")

                text_file.write(f"\nTime:{timer}")
                text_file.write(f"\nnumber of anomalies {str(len(output))}")

                text_file.close()
        else: pass
    cv2.destroyAllWindows()


In [14]:
vid_no = 33
path = f'C:\\Users\ADMIN\Desktop\AIC21-Track4-Anomaly-Detection_full\AIC21-Track4-Anomaly-Detection\\aic21-track4-train-data\{vid_no}.mp4'
deploy_model(path, weights)

vid no: 33
0
0

0: 352x640 5 cars, 3 trucks, 21.0ms
Speed: 4.0ms preprocess, 21.0ms inference, 4.0ms postprocess per image at shape (1, 3, 352, 640)
{}
vid no: 33
1
0

0: 352x640 4 cars, 3 trucks, 26.5ms
Speed: 3.0ms preprocess, 26.5ms inference, 3.5ms postprocess per image at shape (1, 3, 352, 640)
{}
vid no: 33
2
1

0: 352x640 4 cars, 2 trucks, 23.0ms
Speed: 2.3ms preprocess, 23.0ms inference, 5.0ms postprocess per image at shape (1, 3, 352, 640)
{}
vid no: 33
3
2

0: 352x640 4 cars, 3 trucks, 18.1ms
Speed: 3.0ms preprocess, 18.1ms inference, 6.0ms postprocess per image at shape (1, 3, 352, 640)
{}
vid no: 33
4
2

0: 352x640 4 cars, 3 trucks, 32.2ms
Speed: 4.0ms preprocess, 32.2ms inference, 5.5ms postprocess per image at shape (1, 3, 352, 640)
{}
vid no: 33
5
2

0: 352x640 4 cars, 2 trucks, 25.0ms
Speed: 2.0ms preprocess, 25.0ms inference, 4.0ms postprocess per image at shape (1, 3, 352, 640)
{}
vid no: 33
6
3

0: 352x640 4 cars, 2 trucks, 30.0ms
Speed: 3.0ms preprocess, 30.0ms infe

In [ ]:
dir